<a href="https://colab.research.google.com/github/shimshimee/class2021Spring/blob/main/sound.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# source: pure tone(sin wave)의 합
# phasor: pure tone을 만듦
# pulse train: source를 만듦

In [5]:
%matplotlib inline
from matplotlib import pyplot as plt
import IPython.display as ipd
import numpy as np
from scipy.signal import lfilter

In [7]:
theta = np.pi/2       # 파이는 180도, 1/2는 90도
np.sin(theta)

1.0

In [11]:
theta = np.pi*0       # 복소수 i는 1j로 표현
                      # 0 90 180 270 360
np.exp(theta*1j)      # (1,0) (0,1) (-1,0) (0,-1)

(1+0j)

In [19]:
import plotly.graph_objects as go
import numpy as np

amp = 1
sr = 10000
dur = 0.5
freq = 100.0

t = np.arange(1, sr * dur+1)/sr
theta = t * 2*np.pi * freq
c = np.exp(theta*1j)

x, y, z = t[0:300], c.real[0:300], c.imag[0:300]
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z,
                                   mode='markers',
                                   marker=dict(size=3))])
fig.show()